In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
#### ETFs #####

#SHY ETF
relative_path = os.path.join('..', '..',"Data",'Consolidated',"ETF", "SHY.csv")
SHY=pd.read_csv(relative_path,index_col=0)

#IEF ETF
relative_path = os.path.join('..', '..',"Data",'Consolidated',"ETF", "IEF.csv")
IEF=pd.read_csv(relative_path,index_col=0)


#### Alternative Forecasts###

## Two Year Treasury Yield

#Three Month Forecasts
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Forecasts","AlternativeForecasts", "TUY3.csv")
TUY3=pd.read_csv(relative_path,index_col=0)

#Six Month Forecasts
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Forecasts","AlternativeForecasts", "TUY6.csv")
TUY6=pd.read_csv(relative_path,index_col=0)

#Twelve Month Forecasts
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Forecasts","AlternativeForecasts", "TUY12.csv")
TUY12=pd.read_csv(relative_path,index_col=0)

## Ten Year Treasury Yield

#Three Month Forecasts
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Forecasts","AlternativeForecasts", "TYY3.csv")
TYY3=pd.read_csv(relative_path,index_col=0)

#Six Month Forecasts
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Forecasts","AlternativeForecasts", "TYY6.csv")
TYY6=pd.read_csv(relative_path,index_col=0)

#Twelve Month Forecasts
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Forecasts","AlternativeForecasts", "TYY12.csv")
TYY12=pd.read_csv(relative_path,index_col=0)

#### Expert Forecasts###

## Two Year Treasury Yield

#Three Month Forecasts
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Forecasts","ExpertForecasts", "TMTU.csv")
TMTU=pd.read_csv(relative_path,index_col=0)

#Six Month Forecasts
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Forecasts","ExpertForecasts", "SMTU.csv")
SMTU=pd.read_csv(relative_path,index_col=0)

#Twelve Month Forecasts
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Forecasts","ExpertForecasts", "TWTU.csv")
TWTU=pd.read_csv(relative_path,index_col=0)


## Ten Year Treasury Yield

#Three Month Forecasts
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Forecasts","ExpertForecasts", "TMTY.csv")
TMTY=pd.read_csv(relative_path,index_col=0)

#Six Month Forecasts
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Forecasts","ExpertForecasts", "SMTY.csv")
SMTY=pd.read_csv(relative_path,index_col=0)

#Twelve Month Forecasts
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Forecasts","ExpertForecasts", "TWTY.csv")
TWTY=pd.read_csv(relative_path,index_col=0)

#### Individual Expert Forecasts###

## Two Year Treasury Yield

#Three Month Forecasts
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Forecasts","IndividualForecasts","FinalPrediction","TMTUI.csv")
TMTUI=pd.read_csv(relative_path,index_col=0)

#Six Month Forecasts
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Forecasts","IndividualForecasts","FinalPrediction" ,"SMTUI.csv")
SMTUI=pd.read_csv(relative_path,index_col=0)

#Twelve Month Forecasts
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Forecasts","IndividualForecasts","FinalPrediction", "TWTUI.csv")
TWTUI=pd.read_csv(relative_path,index_col=0)


## Ten Year Treasury Yield

#Three Month Forecasts
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Forecasts","IndividualForecasts","FinalPrediction" ,"TMTYI.csv")
TMTYI=pd.read_csv(relative_path,index_col=0)

#Six Month Forecasts
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Forecasts","IndividualForecasts","FinalPrediction" ,"SMTYI.csv")
SMTYI=pd.read_csv(relative_path,index_col=0)

#Twelve Month Forecasts
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Forecasts","IndividualForecasts","FinalPrediction" ,"TWTYI.csv")
TWTYI=pd.read_csv(relative_path,index_col=0)

In [ ]:
TWTYI

In [4]:
def df_prep(alt_forecast,expert_forecast,individual_f,etf,q_ahead):
    alt_forecast[["No-Change(F)","No-Change(g)","Implied Yield","Forward"]]=alt_forecast[["No-Change(F)","No-Change(g)","Implied Yield","Forward",]].shift(-q_ahead)
    etf=etf["Close"]
    expert_forecast.index=expert_forecast["SnapDate"]
    expert_Median=expert_forecast["Median"]
    full_df=pd.concat([alt_forecast,expert_Median,individual_f,etf],join="inner",axis=1)
    return(full_df)

In [5]:
strat2Y3M=df_prep(TUY3,TMTU,TMTUI,SHY,1)
strat2Y6M=df_prep(TUY6,SMTU,SMTUI,SHY,2)
strat2Y12M=df_prep(TUY12,TWTU,TWTUI,SHY,4)

strat10Y3M=df_prep(TYY3,TMTY,TMTYI,IEF,1)
strat10Y6M=df_prep(TYY6,SMTY,SMTYI,IEF,2)
strat10Y12M=df_prep(TYY12,TWTY,TWTYI,IEF,4)

In [ ]:
strat2Y3M

In [7]:
#function that assigns sign change of interest rate
def signassignor(value):
    #Go short if you expect rates to rise
    if value>0:
        return (-1)
    #go long if you expect rates to fall
    elif value<0:
        return (1)
    #don't take a position if you expect rates to stay the same
    else:
        return(0)

In [8]:
def trading_rule_A_1Q(df,maturity):
    ncg=[]
    fut=[]
    fwd=[]
    expert=[]
    individual=[]
    ncg_r=[]
    fut_r=[]
    fwd_r=[]
    expert_r=[]
    individual_r=[]
    i=0
    for index,row in df.iterrows():
        if i==0:
            #Assign wheter you go long or short the ETF
            ncg_pred=signassignor(df.loc[index,"No-Change(g)"]-df.loc[index,maturity])
            fut_pred=signassignor(df.loc[index,"Implied Yield"]-df.loc[index,maturity])
            fwd_pred=signassignor(df.loc[index,"Forward"]-df.loc[index,maturity])
            expert_pred=signassignor(df.loc[index,"Median"]-df.loc[index,maturity])
            individual_pred=signassignor(df.loc[index,"Individual"]-df.loc[index,maturity])
            index_snap=index
            i=i+1
        else:
            #calculate the price difference betwen quarter
            profit=df.loc[index,"Close"]-df.loc[index_snap,"Close"]
            #calculate the reuturn between quarters
            r=np.log(df.loc[index,"Close"]/df.loc[index_snap,"Close"])
            #calculate return and profit based on wheter you were long or short
            ncg.append(ncg_pred*profit)
            fut.append(fut_pred*profit)
            fwd.append(fwd_pred*profit)
            expert.append(expert_pred*profit)
            individual.append(individual_pred*profit)
            ncg_r.append(ncg_pred*r)
            fut_r.append(fut_pred*r)
            fwd_r.append(fwd_pred*r)
            expert_r.append(expert_pred*r)
            individual_r.append(individual_pred*r)
            #go long/short for next quarter
            index_snap=index
            ncg_pred=signassignor(df.loc[index,"No-Change(g)"]-df.loc[index,maturity])
            fut_pred=signassignor(df.loc[index,"Implied Yield"]-df.loc[index,maturity])
            fwd_pred=signassignor(df.loc[index,"Forward"]-df.loc[index,maturity])
            expert_pred=signassignor(df.loc[index,"Median"]-df.loc[index,maturity])
            individual_pred=signassignor(df.loc[index,"Individual"]-df.loc[index,maturity])
    #Create DataFrame with all returns and profits        
    return_dic={"No-Change(g) - Profit":ncg,
                "Future Implied Yield - Profit":fut,
               "Forward - Profit":fwd,
               "Expert Forecast - Profit": expert,
                "Individual Forecast - Profit":individual,
               "No-Change(g) - Return":ncg_r,
                "Future Implied Yield - Return":fut_r,
               "Forward - Return":fwd_r,
               "Expert Forecast - Return": expert_r
               ,"Individual Forecast - Return":individual_r}
    return_df=pd.DataFrame(return_dic, index=df.index[:-1])
    return(return_df)

In [9]:
def trading_rule_A_2Q(df,maturity):
    ncg=[]
    fut=[]
    fwd=[]
    expert=[]
    individual=[]
    ncg_r=[]
    fut_r=[]
    fwd_r=[]
    expert_r=[]
    individual_r=[]
    i=0
    for index,row in df.iterrows():
        if i==0:
            #Assign wheter you go long or short the ETF
            ncg_pred=signassignor(df.loc[index,"No-Change(g)"]-df.loc[index,maturity])
            fut_pred=signassignor(df.loc[index,"Implied Yield"]-df.loc[index,maturity])
            fwd_pred=signassignor(df.loc[index,"Forward"]-df.loc[index,maturity])
            expert_pred=signassignor(df.loc[index,"Median"]-df.loc[index,maturity])
            individual_pred=signassignor(df.loc[index,"Individual"]-df.loc[index,maturity])
            index_snap=index
            i=i+1
        elif i==1:
            #calculate the price difference betwen quarter
            profit=df.loc[index,"Close"]-df.loc[index_snap,"Close"]
            #calculate the reuturn between quarters
            r=np.log(df.loc[index,"Close"]/df.loc[index_snap,"Close"])
            #calculate return and profit based on wheter you were long or short
            ncg.append(ncg_pred*profit)
            fut.append(fut_pred*profit)
            fwd.append(fwd_pred*profit)
            expert.append(expert_pred*profit)
            individual.append(individual_pred*profit)
            ncg_r.append(ncg_pred*r)
            fut_r.append(fut_pred*r)
            fwd_r.append(fwd_pred*r)
            expert_r.append(expert_pred*r)
            individual_r.append(individual_pred*r)
            #Assign wheter you go long or short the ETF
            ncg_pred_1=signassignor(df.loc[index,"No-Change(g)"]-df.loc[index,maturity])
            fut_pred_1=signassignor(df.loc[index,"Implied Yield"]-df.loc[index,maturity])
            fwd_pred_1=signassignor(df.loc[index,"Forward"]-df.loc[index,maturity])
            expert_pred_1=signassignor(df.loc[index,"Median"]-df.loc[index,maturity])
            individual_pred_1=signassignor(df.loc[index,"Individual"]-df.loc[index,maturity])
            index_snap=index
            i=i+1
        else:
            net_position_ncg=ncg_pred+ncg_pred_1
            net_position_fut=fut_pred+fut_pred_1
            net_position_fwd=fwd_pred+fwd_pred_1
            net_position_expert=expert_pred+expert_pred_1
            net_position_individual=individual_pred+individual_pred_1
            #calculate the price difference betwen quarter
            profit=df.loc[index,"Close"]-df.loc[index_snap,"Close"]
            #calculate the reuturn between quarters
            r=np.log(df.loc[index,"Close"]/df.loc[index_snap,"Close"])
            #calculate return and profit based on wheter you were long or short
            ncg.append(net_position_ncg*profit)
            fwd.append(net_position_fwd*profit)
            fut.append(net_position_fut*profit)
            expert.append(net_position_expert*profit)
            individual.append(net_position_individual*profit)
            ncg_r.append(net_position_ncg*r/2)
            fut_r.append(net_position_fut*r/2)
            fwd_r.append(net_position_fwd*r/2)
            expert_r.append(net_position_expert*r/2)
            individual_r.append(net_position_individual*r/2)
            #go long/short for two quarter ahead
            index_snap=index
            ncg_pred=ncg_pred_1
            fut_pred=fut_pred_1
            fwd_pred=fwd_pred_1
            expert_pred=expert_pred_1
            individual_pred=individual_pred_1
            ncg_pred_1=signassignor(df.loc[index,"No-Change(g)"]-df.loc[index,maturity])
            fut_pred_1=signassignor(df.loc[index,"Implied Yield"]-df.loc[index,maturity])
            fwd_pred_1=signassignor(df.loc[index,"Forward"]-df.loc[index,maturity])
            expert_pred_1=signassignor(df.loc[index,"Median"]-df.loc[index,maturity])
            individual_pred_1=signassignor(df.loc[index,"Individual"]-df.loc[index,maturity])
    #Create DataFrame with all returns and profits        
    return_dic={"No-Change(g) - Profit":ncg,
                "Future Implied Yield - Profit":fut,
               "Forward - Profit":fwd,
               "Expert Forecast - Profit": expert,
                "Individual Forecast - Profit":individual,
               "No-Change(g) - Return":ncg_r,
                "Future Implied Yield - Return":fut_r,
               "Forward - Return":fwd_r,
               "Expert Forecast - Return": expert_r
               ,"Individual Forecast - Return":individual_r}
    return_df=pd.DataFrame(return_dic, index=df.index[:-1])
    return(return_df)

In [10]:
def trading_rule_A_4Q(df,maturity):
    ncg=[]
    fut=[]
    fwd=[]
    expert=[]
    individual=[]
    ncg_r=[]
    fut_r=[]
    fwd_r=[]
    expert_r=[]
    individual_r=[]
    i=0
    for index,row in df.iterrows():
        if i==0:
            #Assign wheter you go long or short the ETF
            ncg_pred=signassignor(df.loc[index,"No-Change(g)"]-df.loc[index,maturity])
            fut_pred=signassignor(df.loc[index,"Implied Yield"]-df.loc[index,maturity])
            fwd_pred=signassignor(df.loc[index,"Forward"]-df.loc[index,maturity])
            expert_pred=signassignor(df.loc[index,"Median"]-df.loc[index,maturity])
            individual_pred=signassignor(df.loc[index,"Individual"]-df.loc[index,maturity])
            index_snap=index
            i=i+1
        elif i==1:
            #calculate the price difference betwen quarter
            profit=df.loc[index,"Close"]-df.loc[index_snap,"Close"]
            #calculate the reuturn between quarters
            r=np.log(df.loc[index,"Close"]/df.loc[index_snap,"Close"])
            #calculate return and profit based on wheter you were long or short
            net_position_ncg=ncg_pred
            net_position_fut=fut_pred
            net_position_fwd=fwd_pred
            net_position_expert=expert_pred
            net_position_individual=individual_pred
            #calculate return and profit based on wheter you were long or short
            ncg.append(net_position_ncg*profit)
            fwd.append(net_position_fwd*profit)
            fut.append(net_position_fut*profit)
            expert.append(net_position_expert*profit)
            individual.append(net_position_individual*profit)
            ncg_r.append(net_position_ncg*r)
            fut_r.append(net_position_fut*r)
            fwd_r.append(net_position_fwd*r)
            expert_r.append(net_position_expert*r)
            individual_r.append(net_position_individual*r)
            #Assign wheter you go long or short the ETF
            ncg_pred_1=signassignor(df.loc[index,"No-Change(g)"]-df.loc[index,maturity])
            fut_pred_1=signassignor(df.loc[index,"Implied Yield"]-df.loc[index,maturity])
            fwd_pred_1=signassignor(df.loc[index,"Forward"]-df.loc[index,maturity])
            expert_pred_1=signassignor(df.loc[index,"Median"]-df.loc[index,maturity])
            individual_pred_1=signassignor(df.loc[index,"Individual"]-df.loc[index,maturity])
            index_snap=index
            i=i+1
        elif i==2:
            net_position_ncg=ncg_pred+ncg_pred_1
            net_position_fwd=fwd_pred+fwd_pred_1
            net_position_fut=fut_pred+fut_pred_1
            net_position_expert=expert_pred+expert_pred_1
            net_position_individual=individual_pred+individual_pred_1
            #calculate the price difference betwen quarter
            profit=df.loc[index,"Close"]-df.loc[index_snap,"Close"]
            #calculate the reuturn between quarters
            r=np.log(df.loc[index,"Close"]/df.loc[index_snap,"Close"])
            #calculate return and profit based on wheter you were long or short
            ncg.append(net_position_ncg*profit)
            fut.append(net_position_fut*profit)
            fwd.append(net_position_fwd*profit)
            expert.append(net_position_expert*profit)
            individual.append(net_position_individual*profit)
            ncg_r.append(net_position_ncg*r/2)
            fut_r.append(net_position_fut*r/2)
            fwd_r.append(net_position_fwd*r/2)
            expert_r.append(net_position_expert*r/2)
            individual_r.append(net_position_individual*r/2)
            #Assign wheter you go long or short the ETF
            ncg_pred_2=signassignor(df.loc[index,"No-Change(g)"]-df.loc[index,maturity])
            fut_pred_2=signassignor(df.loc[index,"Implied Yield"]-df.loc[index,maturity])
            fwd_pred_2=signassignor(df.loc[index,"Forward"]-df.loc[index,maturity])
            expert_pred_2=signassignor(df.loc[index,"Median"]-df.loc[index,maturity])
            individual_pred_2=signassignor(df.loc[index,"Individual"]-df.loc[index,maturity])
            index_snap=index
            i=i+1
        elif i==3:
            net_position_ncg=ncg_pred+ncg_pred_1+ncg_pred_2
            net_position_fut=fut_pred+fut_pred_1+fut_pred_2
            net_position_fwd=fwd_pred+fwd_pred_1+fwd_pred_2
            net_position_expert=expert_pred+expert_pred_1+expert_pred_2
            net_position_individual=individual_pred+individual_pred_1+individual_pred_2
            #calculate the price difference betwen quarter
            profit=df.loc[index,"Close"]-df.loc[index_snap,"Close"]
            #calculate the reuturn between quarters
            r=np.log(df.loc[index,"Close"]/df.loc[index_snap,"Close"])
            #calculate return and profit based on wheter you were long or short
            ncg.append(net_position_ncg*profit)
            fut.append(net_position_fut*profit)
            fwd.append(net_position_fwd*profit)
            expert.append(net_position_expert*profit)
            individual.append(net_position_individual*profit)
            ncg_r.append(net_position_ncg*r/3)
            fut_r.append(net_position_fut*r/3)
            fwd_r.append(net_position_fwd*r/3)
            expert_r.append(net_position_expert*r/3)
            individual_r.append(net_position_individual*r/3)
            #Assign wheter you go long or short the ETF
            ncg_pred_3=signassignor(df.loc[index,"No-Change(g)"]-df.loc[index,maturity])
            fut_pred_3=signassignor(df.loc[index,"Implied Yield"]-df.loc[index,maturity])
            fwd_pred_3=signassignor(df.loc[index,"Forward"]-df.loc[index,maturity])
            expert_pred_3=signassignor(df.loc[index,"Median"]-df.loc[index,maturity])
            individual_pred_3=signassignor(df.loc[index,"Individual"]-df.loc[index,maturity])
            index_snap=index
            i=i+1
        else:
            net_position_ncg=ncg_pred+ncg_pred_1+ncg_pred_2+ncg_pred_3
            net_position_fut=fut_pred+fut_pred_1+fut_pred_2+fut_pred_3
            net_position_fwd=fwd_pred+fwd_pred_1+fwd_pred_2+fwd_pred_3
            net_position_expert=expert_pred+expert_pred_1+expert_pred_2+expert_pred_3
            net_position_individual=individual_pred+individual_pred_1+individual_pred_2+individual_pred_3
            #calculate the price difference betwen quarter
            profit=df.loc[index,"Close"]-df.loc[index_snap,"Close"]
            #calculate the reuturn between quarters
            r=np.log(df.loc[index,"Close"]/df.loc[index_snap,"Close"])
            #calculate return and profit based on wheter you were long or short
            ncg.append(net_position_ncg*profit)
            fut.append(net_position_fut*profit)
            fwd.append(net_position_fwd*profit)
            expert.append(net_position_expert*profit)
            individual.append(net_position_individual*profit)
            ncg_r.append(net_position_ncg*r/4)
            fut_r.append(net_position_fut*r/4)
            fwd_r.append(net_position_fwd*r/4)
            expert_r.append(net_position_expert*r/4)
            individual_r.append(net_position_individual*r/4)
            #go long/short for two quarter ahead
            index_snap=index
            ncg_pred=ncg_pred_1
            fut_pred=fut_pred_1
            fwd_pred=fwd_pred_1
            expert_pred=expert_pred_1
            individual_pred=individual_pred_1
            ncg_pred_1=ncg_pred_2
            fut_pred_1=fut_pred_2
            fwd_pred_1=fwd_pred_2
            expert_pred_1=expert_pred_2
            individual_pred_1=individual_pred_2
            ncg_pred_2=ncg_pred_3
            fwd_pred_2=fwd_pred_3
            fut_pred_2=fut_pred_3
            expert_pred_2=expert_pred_3
            individual_pred_2=individual_pred_3
            ncg_pred_3=signassignor(df.loc[index,"No-Change(g)"]-df.loc[index,maturity])
            fut_pred_3=signassignor(df.loc[index,"Implied Yield"]-df.loc[index,maturity])
            fwd_pred_3=signassignor(df.loc[index,"Forward"]-df.loc[index,maturity])
            expert_pred_3=signassignor(df.loc[index,"Median"]-df.loc[index,maturity])
            individual_pred_3=signassignor(df.loc[index,"Individual"]-df.loc[index,maturity])
    #Create DataFrame with all returns and profits        
    #Create DataFrame with all returns and profits        
    return_dic={"No-Change(g) - Profit":ncg,
                "Future Implied Yield - Profit":fut,
               "Forward - Profit":fwd,
               "Expert Forecast - Profit": expert,
                "Individual Forecast - Profit":individual,
               "No-Change(g) - Return":ncg_r,
                "Future Implied Yield - Return":fut_r,
               "Forward - Return":fwd_r,
               "Expert Forecast - Return": expert_r
               ,"Individual Forecast - Return":individual_r}
    return_df=pd.DataFrame(return_dic, index=df.index[:-1])
    return(return_df)

In [11]:
returnsA2Y3M=trading_rule_A_1Q(strat2Y3M,"2Y")
returnsA2Y6M=trading_rule_A_2Q(strat2Y6M,"2Y")
returnsA2Y12M=trading_rule_A_4Q(strat2Y12M,"2Y")

returnsA10Y3M=trading_rule_A_1Q(strat10Y3M,"10Y")
returnsA10Y6M=trading_rule_A_2Q(strat10Y6M,"10Y")
returnsA10Y12M=trading_rule_A_4Q(strat10Y12M,"10Y")

In [12]:
def filesaver(df,docname):
    #Save file using relative path
    relative_path = os.path.join('..', '..',"Data","Results","TradingStrategies","StrategyA","ETF",docname)
    df.to_csv(relative_path)

In [13]:
filesaver(returnsA2Y3M,"returnsA2Y3M.csv")
filesaver(returnsA2Y6M,"returnsA2Y6M.csv")
filesaver(returnsA2Y12M,"returnsA2Y12M.csv")

filesaver(returnsA10Y3M,"returnsA10Y3M.csv")
filesaver(returnsA10Y6M,"returnsA10Y6M.csv")
filesaver(returnsA10Y12M,"returnsA10Y12M.csv")